In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
depression_data = pd.read_csv('processed_data/depression_data.csv')
for c in depression_data.columns.tolist():
    print(c)

SEQN
DPQ010
DPQ020
DPQ030
DPQ040
DPQ050
DPQ060
DPQ070
DPQ080
DPQ090
DPQ100
ACD010A
ACD010B
ACD010C
ACD040
ALQ111
ALQ121
ALQ130
ALQ142
ALQ270
ALQ280
ALQ151
ALQ170
BPQ020
BPQ030
BPQ150
BPQ080
BPQ101D
DBQ010
DBD030
DBD041
DBD050
DBD055
DBD061
DBQ073A
DBQ073B
DBQ073C
DBQ073D
DBQ073E
DBQ073U
DBQ301
DBQ330
DBQ360
DBQ370
DBD381
DBQ390
DBQ400
DBD411
DBQ421
DBQ424
DBQ930
DBQ935
DBQ940
DBQ945
DIQ010
DID040
DIQ160
DIQ180
DIQ050
DID060
DIQ060U
DIQ070
FNQ021
FNQ041
FNQ050
FNQ060
FNQ080
FNQ160
FNQ100
FNQ110
FNQ120
FNQ170
FNQ180
FNQ190
FNQ130
FNQ200
FNQ140
FNQ150
FNDCDI
FNQ410
FNQ430
FNQ440
FNQ450
FNQ460
FNQ470
FNQ480
FNQ490
FNQ510
FNQ520
FNQ530
FNQ540
FNDADI
FNDAEDI
FSD032A
FSD032B
FSD032C
FSD041
FSD052
FSD061
FSD071
FSD081
FSD092
FSD102
FSDAD
FSD151
FSQ165
FSD165N
FSQ012
FSD012N
FSD230
FSD230N
FSD795
FSD225
FSD235
FSD162
FSQ760
FSD760N
FSQ653
FSD660ZC
FSD675
FSD680
FSD670ZC
FSQ690
FSQ695
HIQ011
HIQ032A
HIQ032B
HIQ032C
HIQ032D
HIQ032E
HIQ032F
HIQ032H
HIQ032I
HIQ210
HOD051
HSQ590
HUQ010
HUQ030
HUQ042